<a href="https://colab.research.google.com/github/VictorCioce/VictorCioce/blob/main/LagoaPatos_visualizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Última atualização: 14/01/2022

Este script irá Usando o GEE:
 - Acessa as base de dados do Landsat 5 e 8
 - Selecionar imagens por Orbita 221 cena 081/082, Data, Cobertura de nuvens e Região de interesse das Imagens;
 - Mosaico com as imagens de mesma data em cenas diferentes(81/82);
 - Recorte por uma camada mascara - Lagoa dos Patos.
 - Filtração e Visualização para TOA Images(concluída 22/12/2021):
      A = Landsat 5
      B = Landsat 8


# Instalação e Inicialização do Earth Engine

In [ ]:
# Instalando API do Google Earth Engine para outros serviços do Google
!pip install earthengine-api


In [ ]:
# Importanto o Earth Engine (ee) e autenticando
import ee
ee.Authenticate()
ee.Initialize()


# Declaração das Coordenadas da feição da Lagoa dos Patos - RS

In [ ]:
# Definindo uma área em torno da Lagoa dos Patos - RS
lagoa_geo = ee.Geometry.Polygon([[-50.9847, -30.3578], [-50.9560, -30.3147], 
                             [-50.6642, -30.1425], [-50.5015, -30.1329], 
                             [-50.5207, -30.4582], [-50.6259, -30.5204], 
                             [-50.6977, -30.8218], [-50.9225, -30.9223], 
                             [-50.9321, -31.0371], [-51.1521, -31.1232], 
                             [-51.1378, -31.3863], [-51.6209, -31.8264], 
                             [-51.9797, -31.9029], [-52.0180, -32.2330], 
                             [-52.1615, -32.2091], [-52.3146, -31.9938], 
                             [-52.1567, -31.4485], [-51.5683, -31.0562], 
                             [-51.3052, -30.4199], [-51.3722, -29.9894], 
                             [-51.2526, -29.9655], [-51.1952, -30.1568], 
                             [-51.0134, -30.2525], [-50.9847, -30.3578]])

# Declarando as coordenadas da Lagoa dos Patos como uma Features para recorte
lagoa_recorte = ee.Feature(lagoa_geo)


# Definindo a Função ymdList para visualização de datas

In [ ]:
# Função para visualizar as datas das imagens
def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.Number.parse(image.date().format("YYYYMMdd"));
        newlist = ee.List(newlist);
        return ee.List(newlist.add(date).sort())
    ymd = imgcol.iterate(iter_func, ee.List([]))
    return list(ee.List(ymd).reduce(ee.Reducer.frequencyHistogram()).getInfo().keys())


# Definição da Função Folium para gerar Mapas da Lagoa dos Patos

In [ ]:
# Função para visualização
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Definindo parametros da composição do mapa RGB True Color
lp_rgb_param = {
    'bands': ['B3', 'B2', 'B1'],
    'min': 0,
    'max': 0.5,
    'gamma': [0.95, 1.1, 1]
}

# Defindo o centro do mapa para a Lagos dos Patos por Folium 
map_lagoapatos = folium.Map(location=[-31.15666425, -51.35544453], zoom_start=8)


# 1°Seção A: Filtração e Visualização para TOA Images Landsat 5



Vizualização por USGS Landsat 8 Collection 1 Tier 1 TOA Reflectance

In [ ]:
# Variável para acessar coleções do Landsat 5 (lp_LT05_81)
# Usando filtros de:
#   Data
#   Coordenadas
#   Path & Row: 081
#   Cobertura de nuvens menor de 15%
# Foi removido o filtro:    .filterBounds(ee.Geometry.Polygon([[-52.11, -29.25], [-50.22, -29.25], [-51.02, -32.78], [-52.98, -32.40]]))\

lp_LT05_81 = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")\
    .filterDate('2000-06-23','2011-10-28')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 81))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

lp_LT05_82 = ee.ImageCollection("LANDSAT/LT05/C01/T1_TOA")\
    .filterDate('2000-06-23','2011-10-28')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 82))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

print('Total de imagens Row81: ', lp_LT05_81.size().getInfo())
print('Total de imagens Row82: ', lp_LT05_82.size().getInfo())


In [ ]:
# Usando a função ymdList para visualização de Datas
print('Datas disponíveis ROW 081: ', ymdList(lp_LT05_81))
print('Datas disponíveis ROW 082: ', ymdList(lp_LT05_82))


Datas disponíveis ROW 081:  ['20000623', '20000927', '20010101', '20010525', '20010712', '20010728', '20011219', '20020120', '20020221', '20020309', '20030904', '20031123', '20040211', '20040330', '20040517', '20040704', '20040720', '20040906', '20041008', '20050112', '20050418', '20050504', '20051112', '20060523', '20060608', '20060624', '20061014', '20061115', '20061201', '20070102', '20070203', '20070323', '20070526', '20071001', '20080206', '20080309', '20080325', '20081003', '20081206', '20081222', '20090107', '20090123', '20090328', '20090413', '20090429', '20090702', '20090920', '20100211', '20100822', '20101110', '20110505', '20110910']
Datas disponíveis ROW 082:  ['20000623', '20000927', '20010101', '20010202', '20010407', '20010525', '20010712', '20011117', '20011219', '20020120', '20020309', '20030819', '20030920', '20031123', '20040126', '20040211', '20040501', '20040720', '20040906', '20041024', '20041227', '20050112', '20050128', '20050418', '20050504', '20050621', '20050

In [ ]:
# Criação de um Mosaico e geração de seu mapa
# Declaração das variáveis do mosaico.
lp_LT05_81 = ee.Image("LANDSAT/LT05/C01/T1_TOA/LT05_221081_20010525")
lp_LT05_82 = ee.Image("LANDSAT/LT05/C01/T1_TOA/LT05_221082_20010525")

# Gerando a variável mosaico
lp_LT05_mosaic = ee.ImageCollection([lp_LT05_81, lp_LT05_82]).mosaic()

# Função Clip que contem o recorte do mosaico sobre a área da Lagoa dos Patos -RS
lp_LT05 = lp_LT05_mosaic.clip(lagoa_recorte)

# Add the image layer to the map and display it.
map_lagoapatos.add_ee_layer(lp_LT05, lp_rgb_param, 'true color')
display(map_lagoapatos)


# 1°Seção B: Filtração e Visualização para TOA Images Landsat 8; 

In [ ]:
# Variável para acessar coleções do Landsat 8 (lp_LC08)
# Usando filtros de:
#   Data
#   Coordenadas
#   Path & Row: 081
#   Cobertura de nuvens menor de 15%
# Foi removido o filtro:    .filterBounds(ee.Geometry.Polygon([[-52.11, -29.25], [-50.22, -29.25], [-51.02, -32.78], [-52.98, -32.40]]))\

lp_LC08_81 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")\
    .filterDate('2013-08-30','2020-12-23')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 81))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

lp_LC08_82 = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")\
    .filterDate('2013-08-30','2020-12-23')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 82))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

print('Total de imagens Row81: ', lp_LC08_81.size().getInfo())
print('Total de imagens Row82: ', lp_LC08_82.size().getInfo())

In [ ]:
# Usando a função ymdList para visualização de Datas
print('Datas disponíveis ROW 081: ', ymdList(lp_LC08_81))
print('Datas disponíveis ROW 082: ', ymdList(lp_LC08_82))


In [ ]:
# Criação de um Mosaico e geração de seu mapa
# Declaração das variáveis do mosaico.
lp_LC08_81 = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_221081_20130830")
lp_LC08_82 = ee.Image("LANDSAT/LC08/C01/T1_TOA/LC08_221082_20130830")

# Gerando a variável mosaico
lp_LC08_mosaic = ee.ImageCollection([lp_LC08_81, lp_LC08_82]).mosaic()

# Função Clip para recortar o mosaico pela áreas de interesse
lp_LC08 = lp_LC08_mosaic.clip(lagoa_recorte)

# Add the image layer to the map and display it.
map_lagoapatos.add_ee_layer(lp_LC08, lp_rgb_param, 'true color')
display(map_lagoapatos)


In [ ]:
lp_LT05_81_SR = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
    .filterDate('2000-06-23','2011-10-28')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 81))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

lp_LT05_82_SR = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")\
    .filterDate('2000-06-23','2011-10-28')\
    .filter(ee.Filter.eq('WRS_PATH', 221))\
    .filter(ee.Filter.eq('WRS_ROW', 82))\
    .filterMetadata('CLOUD_COVER', 'less_than', 20)

print('Total de imagens Row81: ', lp_LT05_81_SR.size().getInfo())
print('Total de imagens Row82: ', lp_LT05_82_SR.size().getInfo())

In [ ]:
print('Datas disponíveis ROW 081: ', ymdList(lp_LT05_81_SR))
print('Datas disponíveis ROW 082: ', ymdList(lp_LT05_82_SR))